<a href="https://colab.research.google.com/github/komazawa-deep-learning/komazawa-deep-learning.github.io/blob/master/2024notebooks/2024_0517opencv_Object_Detection_with_Haar_Cascades.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [Object Detection with Haar Cascades in Python](https://towardsdatascience.com/object-detection-with-haar-cascades-in-python-ad9e70ed50aa)

## 理論<!-- ## Theory lesson-->

Haar 特徴に基づくカスケード分類器を用いて，顔，目，笑顔，眼鏡を検出する。
この手法は 2001 年に P. Viola と M. Jones によって提案された[1]。
要するに，カスケード関数と呼ばれるものを，大量の正画像と負画像（正画像には目的の物体が含まれ，負画像にはそれが含まれていないことを意味する）に対して学習させ，それを物体検出に利用する機械学習手法である。
<!-- We are going to use Haar Feature-based Cascade Classifiers to detect faces, eyes, smiles as well as eyeglasses.
The method was proposed by P. Viola and M. Jones in 2001 [1].
In short, it is a machine learning method where a so-called cascade function is trained on a large amount of positive and negative images (positive meaning it includes the desired object and negative images lack it), which in turn can be used for object detection.-->

実際に説明するために，Haar 特徴量の概念を紹介する。
Haar 特徴量は，畳み込みカーネルとして働く下の黒と白のボックスによって得られる。
特徴量は，より具体的には，黒い四角形の下の画素の合計から白い四角形の下の画素の合計を引くことで得られる単一の値である。[2]
<!-- To actually explain it we introduce the concept of Haar features, they are obtained by the black and white boxes below which act as convolutional kernels.
The features are more specifically single values received by subtracting the sum of pixels under the white rectangles from the sum of pixels under the black rectangles. [2] -->

<center>
<img src="https://miro.medium.com/v2/resize:fit:640/format:webp/1*hbFPsfsCqV8rf1MV8b8p5w.jpeg">
</center>

畳み込みという言葉に不安を感じるなら，ウィキペディアの画像を使って簡単に説明しよう。
畳み込みとは，基本的に 2 つの関数が互いに影響を及ぼし合った結果である。
つまり，この場合は，ボックスの白と黒の部分の画素の合計がどのように相互作用するか，つまり，微分されて1つの値が生成されることを意味する。もちろん，領域内の画素の和を効率的に計算するアルゴリズムもあり，それは Summed-Area Table と呼ばれ，1984 年にF. Crow によって発表された[3]。
これは後に画像処理の分野で一般化され，積分画像という名前で呼ばれるようになったが，ここではそれについて深く掘り下げることはしない。
<!-- If you get uneasy about the word convolution, let me simplify it with the help of an image from Wikipedia.
It is essentially the result of two functions affecting one another.
So in our case, it signifies how the sum of pixels in the white and black part of the boxes interact, i.e. are differentiated to produce a single value. Of course, there is also an algorithm for calculating the sum of pixels efficiently inside an area, it’s called summed-area table and was published by F. Crow in 1984 [3].
It was later popularized in the image processing domain and goes under the name integral image, but I won’t delve deeper into that here. -->

<center>
<img src="https://miro.medium.com/v2/resize:fit:640/format:webp/1*CbOUB2WgVOzVRx8iDv5APQ.png">
</center>

次に、想像したとおりに特徴選択が行われる。
さまざまな Haar 特徴を試し，黒と白の矩形間の画素の総和の差について，どれが最大の値を出すかを見る。
最適な Haar 特徴が見つかった例を以下に示す。
通常，目は少し暗く，その下は明るいため，上部が黒，下部が白の横長の長方形が適している。
次に，鼻筋は目よりも明るいことが多いので，真ん中に縦長の白いボックスがある Haar 特徴が適している。
<!-- Next, the feature selection happens as you imagine it would.
You try the different Haar features and see which of those produce the largest value for the difference between the sums of pixels between the black and white rectangles.
We have below an example where optimal Haar features have been found.
The eyes are usually a bit darker whereas the area below likely is lighter, and thus a horizontal rectangle with black up top and white below is suitable.
Secondly, the bridge of the nose is often lighter than the eyes and as such a Haar feature with a vertical white box in the middle is the way to go. -->

<center>
<img src="https://miro.medium.com/v2/resize:fit:640/format:webp/1*64MTUF8nuEvSgBvYmOfhKA.png">
</center>

Haar 特徴という名前は少し奇妙に聞こえるが，実は Haar ウェーブレットと直感的に似ていることに由来する：
<!-- The name Haar features sounds a bit odd, but it actually originates from the intuitive similarity with Haar wavelets which are these bad boys: -->

<center>
<img src="https://miro.medium.com/v2/resize:fit:440/format:webp/1*MUeF9CIalU87NC-6T7mNWw.png">
</center>

これらは直接使われることはないが，特徴（黒と白のボックス）は Haar ライクと呼べるものだ。
このような Haar のような特徴をたくさん画像に適用し，学習画像を正しく分類するための最適な閾値を見つける Adaboost アルゴリズムを使うことができる。
しかし，そのうちの 1 つをどこかに配置すると，たとえ最適な位置に配置したとしても，正事例と負事例内の画像はすべて互いに異なるため，やはり多少の誤差が生じる。
最終的に，最も誤差の少ない Haar 特徴が分類子として選ばれる。
<!-- They are not directly used but the features (black and white boxes) are what we can call Haar-like in the sense that, well you get it by now.
Lots of these Haar-like features can be applied to an image and using the Adaboost algorithm which finds an optimal threshold for classifying the training images correctly.
But placing one of those somewhere, even on the best possible position, will result in some error still since all images within the positive and negative sets differ from each other.
In the end, the Haar features with the smallest error rates are chosen as classifiers.-->

最終的な特徴数は，対策を講じたにもかかわらず非常に多くなる可能性があるため，発明者たちは分類器のカスケードという概念を導入した (これで Haar Feature-based Cascade Classifiers という名前全体がわかった)。
これは，検出を行うときに使われるもので，たくさんの特徴量を使って検出を行うと時間がかかるため，代わりに検出を行うときに，分類器は特徴量のカスケードで構成される。
つまり，最初の Haar 特徴量は (顔検出の場合) 画像が顔の可能性があるかどうかをチェックするだけで，次の段階では，さらにいくつかの最も本質的な Haar 特徴量を持つ。
これは好ましい方法である。
なぜなら，目的の物体を含まない画像は早い段階で破棄され，それ以上処理されないからである。
画像にすべての特徴を一度に投入するのと比較すれば，その利点がわかるだろう。[2]
<!--The final number of features can be quite large despite the measures taken, so the inventors introduced the concept of Cascade of Classifiers (now we have got the whole name Haar Feature-based Cascade Classifiers down by the way).
This is used when doing the detection since it would be slow to do it with lots of features, instead, the classifier consists of a cascade of features when detecting.
So the initial Haar feature might just check if the image could possibly be a face (in the case of face detection), then the following stages have a few more of the most essential Haar features.
This is a favorable method since early on images not containing the desired object are discarded and not processed anymore.
Compare this to throwing all features at the image at once and you see the gain. [2] -->


## 実装<!-- ## Implementation-->

Python で実装する部分に移ろう。
もし，自分のマシンで一緒に追ったり，クールなものを試してみたいなら，[GitHubリポジトリ](https://github.com/deinal/opencv-recognition-demo)や[Google Colab](https://colab.research.google.com/drive/1H2XBOI31j4idgAqyY-brunwOlIajeFRt#scrollTo=KjxUh493oYf-) で公開されている。
後者の方は，残念ながら，ビデオの物体検出はできないが。
とにかく，まずはいくつかのインポートを行い，準備は万端だ。
<!--On to the part where we implement it in Python.
If you want to follow along on your own machine or try out any of the cool stuff it is publicly available in a [GitHub repository](https://github.com/deinal/opencv-recognition-demo) or at [Google Colab](https://colab.research.google.com/drive/1H2XBOI31j4idgAqyY-brunwOlIajeFRt#scrollTo=KjxUh493oYf-), although the latter one can’t, unfortunately, do object detection in videos…
Anyway, we start by doing a few imports so we are all set to go. -->


**OpenCV Demo**

In [2]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
plt.rcParams['figure.figsize'] = [10, 20]

# Load image

In [ ]:
#!wget "https://raw.githubusercontent.com/hd4niel/opencv-recognition-demo/master/people-taking-group-picture.jpg" -O group.jpg
#!wget "https://github.com/deinal/opencv-recognition-demo/images/people-taking-group-picture.jpg" -O group.jpg
#!wget https://github.com/deinal/opencv-recognition-demo/blob/master/images/people-taking-group-picture.jpg -O group.jpg
!wget -O group.jpg https://mingle.jp/wp-content/uploads/2021/03/5060Tokyo.jpg

In [ ]:
#from PIL import Image
#img = Image.open('group.jpg')
img = cv2.imread('group.jpg')
#plt.imshow(img)
#plt.show()

imgrgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(imgrgb)
plt.show()

# Load Haar cascades

In [ ]:
!wget "https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml" -O haarcascade_frontalface_default.xml
!wget "https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_eye.xml" -O haarcascade_eye.xml
!wget "https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_eye_tree_eyeglasses.xml" -O haarcascade_eye_tree_eyeglasses.xml
!wget "https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_smile.xml" -O haarcascade_smile.xml

# Do object detection

In [6]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')
glasses_cascade = cv2.CascadeClassifier('haarcascade_eye_tree_eyeglasses.xml')
smile_cascade = cv2.CascadeClassifier('haarcascade_smile.xml')

img = cv2.imread('group.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

ここからが本題だ。
まず最初に、分類はグレースケール・モードで行われる。
次に，物体検出を行い，矩形のリストとして返す detectMultiScale を見てみよう。
このメソッドにはいくつかのパラメータがあり，好みに応じて調整できる。
<!-- So here is the juicy part I suppose.
First off, classification happens in grayscale mode, and then we can take a look at detectMultiScale which performs the object detection and return them as a list of rectangles.
The method can take a few parameters which can be tweaked to one's liking. -->

* **scaleFactor**： 各画像スケールで画像サイズをどれだけ縮小するかを指定するパラメータ。
この値を大きくすると，いくつかの物体を見逃すリスクはあるが，検出が速くなる。
* **minNeighbors**： 各候補矩形を保持するために，いくつの近傍を持つべきかを指定するパラメータ。
値が高いほど検出は少なくなるが，品質は高くなる。
* **minSize**： 最小物体サイズ。これより小さい物体は無視される。

<!-- * **scaleFactor**: Parameter specifying how much the image size is reduced at each image scale.
Increasing it leads to faster detection with the risk of missing some objects, whereas a small value might sometimes be too thorough.
* **minNeighbors**: Parameter specifying how many neighbors each candidate rectangle should have to retain it.
Higher value results in less detection but with higher quality.
* **minSize**: Minimum possible object size. Objects smaller than that are ignored.
* **maxSize**: Maximum possible object size. Objects larger than that are ignored. -->

私のコードでは、scaleFactorとminNeighborsをいじっている。
例えば、眼鏡は検出しにくかった。おそらく、私が使っている写真に写っている人たちは上を向いていて、入力カスケードが学習したものとは似ていないからだろう。
そのため、scaleFactorとminNeighborsを下げて精度を上げた。
それとは別に、笑顔が多すぎたので、両方のパラメーターを増やした。
<!-- I fiddle with the scaleFactor and minNeighbors in my code.
Eyeglasses, for example, were hard to detect, maybe because the people in the picture I am using are looking upwards and it does not resemble what the input cascades are trained for.
I, therefore, lowered the scaleFactor as well as minNeighbors for more precision.
Apart from that, I detected too many smiles — so I increased both of the parameters there. -->

In [ ]:
# default MultiScale paramters: scaleFactor=1.1, minNeighbors=3
faces = face_cascade.detectMultiScale(gray, 1.3, 5)
for(x, y, w, h) in faces:
    img = cv2.rectangle(img, (x,y), (x+w,y+h), (255,0,0), 3)
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = img[y:y+h, x:x+w]

    smiles = smile_cascade.detectMultiScale(roi_gray, minNeighbors=20)
    for(sx, sy, sw, sh) in smiles:
        cv2.rectangle(roi_color, (sx,sy), (sx+sw,sy+sh), (0,0,255), 3)

    glasses = glasses_cascade.detectMultiScale(roi_gray, scaleFactor=1.04, minNeighbors=1)
    for(gx, gy, gw, gh) in glasses:
        cv2.rectangle(roi_color, (gx,gy), (gx+gw,gy+gh), (0,255,0), 2)

    eyes = eye_cascade.detectMultiScale(roi_gray)
    for(ex, ey, ew, eh) in eyes:
        cv2.rectangle(roi_color, (ex,ey), (ex+ew,ey+eh), (0,255,255), 3)

plt.xticks([])
plt.yticks([])

face_patch = mpatches.Patch(color='blue', label='Faces')
smile_patch = mpatches.Patch(color='red', label='Smiles')
eye_patch = mpatches.Patch(color='yellow', label='Eyes')
glass_patch = mpatches.Patch(color='green', label='Glasses')
plt.legend(handles=[face_patch, smile_patch, eye_patch, glass_patch],
           loc='lower right', fontsize=12)

imgplot = plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

# Try on a webcam photo

In [8]:
# Source: https://colab.research.google.com/notebooks/snippets/advanced_outputs.ipynb#scrollTo=2viqYx97hPMi
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))

  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

In [10]:
img = cv2.imread(filename)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [ ]:
faces = face_cascade.detectMultiScale(gray, 1.3, 5)
for (x,y,w,h) in faces:
    img = cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = img[y:y+h, x:x+w]

    eyes = eye_cascade.detectMultiScale(roi_gray)
    for (ex,ey,ew,eh) in eyes:
        cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)

    smiles = smile_cascade.detectMultiScale(roi_gray, scaleFactor=1.8, minNeighbors=20)
    for (sx,sy,sw,sh) in smiles:
        cv2.rectangle(roi_color,(sx,sy),(sx+sw,sy+sh),(0,0,255),2)

plt.grid(None)
plt.xticks([])
plt.yticks([])
imgplot = plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))